# Documentation - Links from ODS

### API Overview
https://digital.nhs.uk/services/organisation-data-service/apis-for-the-organisation-data-service#section-4

### Search endpoint
https://digital.nhs.uk/services/organisation-data-service/guidance-for-developers/search-endpoint

Search parameters
https://digital.nhs.uk/services/organisation-data-service/guidance-for-developers/search-parameters#name-substring-search-

### Ogranisation endpoint 
https://digital.nhs.uk/services/organisation-data-service/guidance-for-developers/organisation-endpoint

Org succession explanation
https://digital.nhs.uk/services/organisation-data-service/guidance-for-developers/how-to-understand-organisation-succession

More endpoints
https://digital.nhs.uk/services/organisation-data-service/guidance-for-developers/ods-ord-api-endpoints

Implementation guide and links
https://directory.spineservices.nhs.uk/ODSAPISuite

In [2]:
import pandas as pd
import requests
import json

In [3]:
def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

In [4]:
#example search endpoint
response = requests.get("https://directory.spineservices.nhs.uk/ORD/2-0-0/organisations?PostCode=LS14BT")

In [11]:
#example organisation endpoint
ODScode = "504"
response = requests.get("https://directory.spineservices.nhs.uk/ORD/2-0-0/organisations/"+ODScode+"?")

In [12]:
#preview API response
jprint(response.json())

{
    "Organisation": {
        "Date": [
            {
                "Start": "1980-01-01",
                "Type": "Operational"
            },
            {
                "End": "2021-03-31",
                "Start": "1980-01-01",
                "Type": "Legal"
            }
        ],
        "GeoLoc": {
            "Location": {
                "AddrLn1": "THE GUILDHALL",
                "AddrLn2": "ST GILES SQUARE",
                "Country": "ENGLAND",
                "PostCode": "NN1 1DE",
                "Town": "NORTHAMPTON",
                "UPRN": 10091820507
            }
        },
        "LastChangeDate": "2021-04-06",
        "Name": "NORTHAMPTONSHIRE COUNTY COUNCIL",
        "OrgId": {
            "assigningAuthorityName": "HSCIC",
            "extension": "504",
            "root": "2.16.840.1.113883.2.1.3.2.4.18.48"
        },
        "Rels": {
            "Rel": [
                {
                    "Date": [
                        {
                       

In [19]:
data = response.json()

In [9]:
#format whole json reply as normalised dataframe
df = pd.DataFrame.from_dict(pd.json_normalize(response.json()), orient='columns')
df

,Organisation.Name,Organisation.Date,Organisation.OrgId.root,Organisation.OrgId.assigningAuthorityName,Organisation.OrgId.extension,Organisation.Status,Organisation.LastChangeDate,Organisation.orgRecordClass,Organisation.GeoLoc.Location.AddrLn1,Organisation.GeoLoc.Location.Town,Organisation.GeoLoc.Location.PostCode,Organisation.GeoLoc.Location.Country,Organisation.GeoLoc.Location.UPRN,Organisation.Roles.Role,Organisation.Rels.Rel,Organisation.Succs.Succ
0,NHS DIGITAL,"[{'Type': 'Operational', 'Start': '2013-01-21'...",2.16.840.1.113883.2.1.3.2.4.18.48,HSCIC,X26,Active,2019-01-25,RC1,1 TREVELYAN SQUARE,LEEDS,LS1 6AE,ENGLAND,72666252,"[{'id': 'RO116', 'uniqueRoleId': 166234, 'prim...","[{'Date': [{'Type': 'Operational', 'Start': '2...","[{'uniqueSuccId': 36589, 'Date': [{'Type': 'Le..."


In [23]:
#create empty dataframe
df_succ = pd.DataFrame(columns = ["SiteODScode", "SuccessorODScode", "PredecessorODScode", "Date"])

#populate dataframe row by row
for suc in data["Organisation"]["Succs"]["Succ"]:
    site = str(data["Organisation"]["OrgId"]["extension"])
    typesp= (suc["Type"])
    if typesp == "Predecessor":
        successor = "-"
        predecessor = (suc["Target"]["OrgId"]["extension"])
    elif typesp == "Successor":
        successor = (suc["Target"]["OrgId"]["extension"])
        predecessor = "-"
    else:
        successor = "-"
        predecessor = "-"
    
    date = (suc["Date"][0]["Start"])

    df_succ = df_succ.append({'SiteODScode' : site , 
                          'SuccessorODScode' : successor, 
                          'PredecessorODScode' :predecessor, 
                          'Date' : date}, ignore_index=True)
                          
#view dataframe
df_succ

,SiteODScode,SuccessorODScode,PredecessorODScode,Date
0,504,-,V258,2013-04-01
1,504,Z9D4Z,-,2021-04-01
2,504,U6Q5Z,-,2021-04-01
